In [2]:
from typing import Dict, List, Tuple

import tqdm

import numpy as np
import pandas as pd

import scipy
import scipy.stats

import seaborn as sns
import matplotlib.pyplot as plt

import pathlib
import tttrlib

import os

from feda_tools import twodim_hist as tdh
from feda_tools import utilities as utils
from feda_tools import analysis as an

from decimal import Decimal, getcontext

import numpy.ma as ma
from scipy.stats import norm
from scipy.stats import halfnorm

In [3]:
# Load PTU Files
# file_path = pathlib.Path('//130.127.188.19/projects/FoxP_FKH-DNA/20220314_FoxP1_data_all_NK/20220310_V78C_monomer_NK/V78C_monomer_FoxP1_1hr/burstwise_All 0.2027#30')
# bid_path = pathlib.Path('//130.127.188.19/projects/FoxP_FKH-DNA/20220314_FoxP1_data_all_NK/20220310_V78C_monomer_NK/V78C_monomer_FoxP1_1hr/burstwise_All 0.2027#30/BIDs_30ph')

### for testing purposes ###
file_path = pathlib.Path('C:/Users/2administrator/Documents/source/repos/feda_tools/test data/2022/03_02_22_Troubleshooting_detection_efficiencies/Combined_old_thresholds/Split_After_Adjust_HF_54000s_pinhole6-000000.ptu')

#total time 816.9 seconds for this file
file_p ='C:/Users/2administrator/Documents/source/repos/feda_tools/test data/2022/03_02_22_Troubleshooting_detection_efficiencies/Combined_old_thresholds/Split_After_Adjust_HF_54000s_pinhole6-000000.ptu'

In [4]:
data = tttrlib.TTTR(file_p, 'PTU')

In [5]:
all_macro_times = data.macro_times
all_micro_times = data.micro_times
routing_channels = data.routing_channels

In [6]:
#in seconds. usually the first plots are in ms to see the bursts.
macro_res =data.get_header().macro_time_resolution

In [7]:
micro_res = data.get_header().micro_time_resolution

In [8]:
#iterate through macro and micro times to calculate delta time between photon events
arr_size = len(all_macro_times) - 1
photon_time_intervals = np.zeros(arr_size, dtype = np.float64)
lw = 0.25
for i in range(0, len(photon_time_intervals)):
    photon_1 = (all_macro_times[i]*macro_res) + (all_micro_times[i]*micro_res)
    photon_2 = (all_macro_times[i+1]*macro_res) + (all_micro_times[i+1]*micro_res)
    photon_time_intervals[i] = (photon_2 - photon_1)*1000

# create photon ID array
photon_ids = np.arange(1, arr_size + 1)


In [9]:
print(photon_time_intervals[-1])

2.3073616332567326


In [10]:
# plot interactive
%matplotlib qt

# raw data, no running average
plt.plot(photon_ids, np.log(photon_time_intervals), linewidth = 0.25)
plt.xlim(0, 91000)
plt.show()

In [11]:
# Calculate the running average
cumulative_sum = np.cumsum(photon_time_intervals)
running_average = cumulative_sum / np.arange(1, len(photon_time_intervals) + 1)

In [12]:
plt.plot(photon_ids, running_average)

In [13]:
def running_average(data, window_size):
    window = np.ones(window_size) / window_size
    return np.convolve(data, window, mode='valid')

# Create a NumPy array
data = photon_time_intervals

# Set the window size for the running average
window_size = 30

# Calculate the running average
running_avg = running_average(data, window_size)



In [14]:
# Plot the running average
# plt.plot(data, label='Original Data')
lw = 0.25
xarr = np.arange(window_size - 1, len(data))
logrunavg = np.log(running_avg)
plt.plot(xarr, logrunavg, label='Running Average', linewidth = lw)
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.xlim(0, 91000)
plt.show()

In [15]:
# plot the running average as a 2D histogram with 1D histograms on the margins

bins = {"x":141, "y": 141}
xrange = {"min" : 0, "max" : 91000}
yrange = {"min" : -6, "max" : 2}
fig, ax, twodimdata = tdh.make_plot(xarr, logrunavg, "x", "y",xrange ,yrange, bins)

In [16]:

# set the threshold based on visual inspection
threshold_value = -.34

# compress the filtered data to remove the masked values
filtered_logrunavg = ma.masked_less(logrunavg, threshold_value).compressed()

# Set all masked values to zero
counts_logrunavg, bins_logrunavg, _ = plt.hist(logrunavg, bins = bins['y'], alpha=0.6, color='r')
plt.hist(filtered_logrunavg, bins = bins_logrunavg, alpha=0.6, color='b')
plt.show()

In [22]:
# fit with halfnorm
mu, std = halfnorm.fit(filtered_logrunavg)

# counts_logrunavg, bins_logrunavg, _ = plt.hist(logrunavg, bins = bins['y'], density= True, alpha=0.6, color='r')
plt.hist(filtered_logrunavg, bins = bins['y'], density = True, alpha=0.6, color='r')

# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = halfnorm.pdf(x, mu, std)

plt.plot(x, p, 'k', linewidth=2)
title = "Fit Values: {:.2f} and {:.2f}".format(mu, std)
plt.title(title)

# Display the plot
plt.show()

In [23]:
# select photon events that are less likely to be noise based on analysis in previous cell
threshold_value = mu - 2*std
filtered_values = ma.masked_greater(logrunavg, threshold_value)

In [24]:
# Plot the running average and the threshold values

# plt.plot(xarr, logrunavg, label='Running Average', linewidth = lw)
plt.plot(xarr, logrunavg, label='Running Average', linestyle='None', marker = 'o', markersize = 5)
plt.plot(xarr, filtered_values, label='Threshold Values', linestyle='None', marker = '.', markersize = 5)
plt.xlabel('Photon Event #')
plt.ylabel('log(Photon Interval Time)')
plt.legend()
plt.xlim(0, 91000)
plt.show()

In [48]:
### define function for extracting the unmasked segments from the thresholded data.
def extract_unmasked_indices(masked_array):
    unmasked_indices_lists = []
    current_indices = []

    # iterate through masked array and collect unmasked index segments
    for i, value in enumerate(masked_array):
        if np.ma.is_masked(value):
            if current_indices:
                unmasked_indices_lists.append(current_indices)
                current_indices = []
        else:
            current_indices.append(i)

    # handle the last segment
    if current_indices:
        unmasked_indices_lists.append(current_indices)

    return unmasked_indices_lists

In [50]:
### Get a burst index. Each list is a burst, and each list contains the indices of 
### the photon events in the original data.
burst_index = extract_unmasked_indices(filtered_values)

In [59]:
### Duration calc
lp_time = all_macro_times[burst_index[0][-1]] + all_micro_times[[0][-1]]
fp_time = all_macro_times[burst_index[0][0]] + all_micro_times[[0][0]]